In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import glob
import shutil
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.visualize as visualize

notebook_name = "gridrad_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2010-01-20T18:00:00"
end = "2010-01-20T19:00:00"

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
gridrad_options = data.gridrad.gridrad_data_options(
    start=start, end=end, converted_options=converted_options
)
era5_pl_options = data.era5.data_options(start=start, end=end)
era5_sl_options = data.era5.data_options(
    start=start, end=end, data_format="single-levels"
)
data_options = option.consolidate_options(
    [gridrad_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, filename="gridrad_era5")

# Create the grid_options dictionary using the first file in the cpol dataset
grid_options = grid.create_options(
    name="geographic", regrid=False, altitude_spacing=None, geographic_spacing=None
)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, filename="gridrad_geographic")

# Create the track_options dictionary
track_options = option.mcs(
    dataset="gridrad",
    global_flow_margin=70,
    unique_global_flow=False,
)
# Disable tracking/matching for cell and anvil objects
track_options[0]["cell"]["tracking"] = {"method": None}
track_options[0]["anvil"]["tracking"] = {"method": None}
option.check_options(track_options)
option.save_track_options(track_options, filename="gridrad_mcs")

# Create the display_options dictionary
cell_vis_options = visualize.option.runtime_options(
    "cell", save=True, style="presentation", figure_types=["mask"]
)
anvil_vis_options = visualize.option.runtime_options(
    "anvil", save=True, style="presentation", figure_types=["mask"]
)
mcs_vis_options = visualize.option.runtime_options(
    "mcs", save=True, style="presentation", figure_types=["mask", "match"]
)
visualize_options = {
    "cell": cell_vis_options,
    "anvil": anvil_vis_options,
    "mcs": mcs_vis_options,
}
# visualize_options = None
visualize.option.save_display_options(visualize_options, filename="runtime_mcs")

2024-09-23 12:19:41,933 - thor.data.era5 - WARNING - No longitude range provided. Setting to [-90, 90].
2024-09-23 12:19:41,943 - thor.data.era5 - WARNING - No longitude range provided. Setting to [-180, 180].
2024-09-23 12:19:41,944 - thor.data.era5 - WARNING - No longitude range provided. Setting to [-90, 90].
2024-09-23 12:19:41,945 - thor.data.era5 - WARNING - No longitude range provided. Setting to [-180, 180].
2024-09-23 12:19:41,947 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/data_options/gridrad_era5.yaml
2024-09-23 12:19:41,956 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-09-23 12:19:41,958 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/grid_options/gridrad_geographic.yaml
2024-09-23 12:19:41,961 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/track_options/gridrad_mcs.yaml
2024-09-23 12:19:42,002 - thor.option - DEBUG - Saving options to /home/ewan/THOR_outpu

In [3]:
tag_options = None

output_directory = base_local / "runs/gridrad_demo"
if output_directory.exists():
    shutil.rmtree(output_directory)
times = data.utils.generate_times(data_options["gridrad"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=base_local / "runs/gridrad_demo",
)


2024-09-23 12:19:47,361 - thor.track - INFO - Beginning simultaneous tracking.
2024-09-23 12:19:47,473 - thor.track - INFO - Processing 2010-01-20T18:00:00.
2024-09-23 12:19:47,474 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-20T18:00:00.
2024-09-23 12:19:47,476 - thor.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100120T180000Z.nc
2024-09-23 12:19:47,576 - thor.data.gridrad - DEBUG - Filtering GridRad data
2024-09-23 12:19:48,828 - thor.data.gridrad - DEBUG - Removing clutter from the GridRad data
2024-09-23 12:19:48,886 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-09-23 12:19:49,132 - thor.data.gridrad - DEBUG - Removing low level clutter from the GridRad data
2024-09-23 12:19:49,366 - thor.data.gridrad - DEBUG - Removing speckles from the GridRad data
2024-09-23 12:19:49,595 - thor.data.gridrad - DEBUG - Restructuring GridRad dataset.
2024-09-23 12:19:50,476 - thor.data.era5 - INFO - Updating era5_pl dataset fo

KeyboardInterrupt: 